# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

In [63]:
def format_preco(linha):
    if linha['Moeda'] == 'Euro':
        return "€{:,.2f}".format(linha['Preço Original'])
        
    if linha['Moeda'] == 'Dólar':
        return "${:,.2f}".format(linha['Preço Original'])
    
    if linha['Moeda'] == 'Ouro':
        return "{:,.2f}g".format(linha['Preço Original'])

In [64]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# abrir navegador do chrome em 2º plano
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True 
navegador = webdriver.Chrome(options=chrome_options)

navegador.get("https://www.google.com/")

# Passo 1: Pegar a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

C:\Users\Gustavo\AppData\Local\Temp\ipykernel_11296\3380879284.py:8: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


4.9444
5.431492621
323.90


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [68]:
# Passo 4: Importar a lista de produtos
import pandas as pd

df = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,R$999.99,Dólar,4.944500,"R$4,944.45",1.40,"R$6,922.23"
1,Carro Renault,"R$4,500.00",Euro,5.431459,"R$24,441.57",2.00,"R$48,883.13"
2,Notebook Dell,R$899.99,Dólar,4.944500,"R$4,450.00",1.70,"R$7,565.00"
3,IPhone,R$799.00,Dólar,4.944500,"R$3,950.66",1.70,"R$6,716.11"
4,Carro Fiat,"R$3,000.00",Euro,5.431459,"R$16,294.38",1.90,"R$30,959.32"
5,Celular Xiaomi,R$480.48,Dólar,4.944500,"R$2,375.73",2.00,"R$4,751.47"
6,Joia 20g,R$20.00,Ouro,323.810000,"R$6,476.20",1.15,"R$7,447.63"


- Atualizando os preços e o cálculo do Preço Final

In [66]:
# Passo 5: Recalcular o preço de cada produto

df.loc[df["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
df.loc[df["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
df.loc[df["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
df["Preço de Compra"] = df["Preço Original"] * df["Cotação"]

# atualizar o preço final (preço base reais * Margem)
df["Preço de Venda"] = df["Preço de Compra"] * df["Margem"]


df["Cotação"] = df["Cotação"].map("R${:,.2f}".format)

for i, linha in df.iterrows():
    df.loc[i, 'Preço Original'] = format_preco(linha)
df["Preço de Compra"] = df["Preço de Compra"].map("R${:,.2f}".format)
df["Preço de Venda"] = df["Preço de Venda"].map("R${:,.2f}".format)

display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,$999.99,Dólar,R$4.94,"R$4,944.35",1.40,"R$6,922.09"
1,Carro Renault,"€4,500.00",Euro,R$5.43,"R$24,441.72",2.00,"R$48,883.43"
2,Notebook Dell,$899.99,Dólar,R$4.94,"R$4,449.91",1.70,"R$7,564.85"
3,IPhone,$799.00,Dólar,R$4.94,"R$3,950.58",1.70,"R$6,715.98"
4,Carro Fiat,"€3,000.00",Euro,R$5.43,"R$16,294.48",1.90,"R$30,959.51"
5,Celular Xiaomi,$480.48,Dólar,R$4.94,"R$2,375.69",2.00,"R$4,751.37"
6,Joia 20g,20.00g,Ouro,R$323.90,"R$6,478.00",1.15,"R$7,449.70"


### Agora vamos exportar a nova base de preços atualizada

In [67]:
# Passo 6: Salvar os novos preços dos produtos
df.to_excel("Produtos Novo.xlsx", index=False)